<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/prueba_usuarios_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simulación de un chat con memoria de usuario

# Memoria para almacenar el contexto por usuario (simulación en memoria)
user_sessions = {}

# Función para manejar la interacción con el usuario
def chat_with_user(user_id, message):
    # Recuperar el contexto del usuario si existe
    if user_id not in user_sessions:
        user_sessions[user_id] = {"context": []}
        print(f"[INFO] Nueva sesión creada para el usuario: {user_id}")

    context = user_sessions[user_id]["context"]

    # Crear la respuesta basándonos en el contexto
    response = f"Hola, {user_id}. Me dijiste: {message}. "
    if context:
        response += f"Recuerdo que también mencionaste: {' | '.join(context)}."
    else:
        response += "Es nuestra primera conversación."

    # Actualizar el contexto con el mensaje actual
    context.append(message)
    user_sessions[user_id]["context"] = context

    return response

# Probar el sistema de chat
print("Prueba de Chat con Memoria")
while True:
    user_id = input("Ingresa tu ID de usuario (escribe 'salir' para terminar): ")
    if user_id.lower() == "salir":
        break
    message = input("Tú: ")
    response = chat_with_user(user_id, message)
    print(f"Chatbot: {response}")


Prueba de Chat con Memoria
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: David
[INFO] Nueva sesión creada para el usuario: 1
Chatbot: Hola, 1. Me dijiste: David. Es nuestra primera conversación.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: David
Chatbot: Hola, 1. Me dijiste: David. Recuerdo que también mencionaste: David.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: Hola
Chatbot: Hola, 1. Me dijiste: Hola. Recuerdo que también mencionaste: David | David.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 2
Tú: Bea
[INFO] Nueva sesión creada para el usuario: 2
Chatbot: Hola, 2. Me dijiste: Bea. Es nuestra primera conversación.
Ingresa tu ID de usuario (escribe 'salir' para terminar): salir


In [2]:
user_sessions

{'1': {'context': ['David', 'David', 'Hola']}, '2': {'context': ['Bea']}}

## Nuevo enfoque con API DE MISTRAL


In [5]:
from getpass import getpass

# Solicitar la API key de Mistral
MISTRAL_API_KEY = getpass("Ingresa tu API key de Mistral: ")

# Confirmar que la clave fue ingresada (sin mostrarla)
if MISTRAL_API_KEY:
    print("API key configurada correctamente.")
else:
    print("No ingresaste una API key. Por favor, reinicia y vuelve a intentarlo.")


Ingresa tu API key de Mistral: ··········
API key configurada correctamente.


In [13]:
import requests
from flask import Flask, request, jsonify
from threading import Thread
from getpass import getpass

# Solicitar la API key de Mistral
MISTRAL_API_KEY = getpass("Ingresa tu API key de Mistral: ")

if not MISTRAL_API_KEY:
    raise ValueError("No se ingresó una API key. Deteniendo ejecución.")

# Simulación de sesiones en memoria
user_sessions = {}

# URL de la API de Mistral
MISTRAL_API_URL = "https://api.mistral.ai/v1/chat/completions"

app = Flask(__name__)

def query_mistral(prompt):
    """
    Consulta la API de Mistral para generar una respuesta.
    """
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "mistral-7b-v0.1",  # Cambia este modelo según la documentación
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 150,
        "temperature": 0.7,  # Creatividad de la respuesta
        "top_p": 1.0  # Diversidad en la generación
    }

    print("Payload enviado:", payload)  # Ayuda para depurar el payload enviado

    try:
        response = requests.post(MISTRAL_API_URL, json=payload, headers=headers)
        response.raise_for_status()  # Lanza excepción para códigos de error HTTP
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        # Capturar errores HTTP o de red
        print("Respuesta completa del error:", e.response.text if e.response else "Sin respuesta del servidor")
        return f"Error al consultar la API de Mistral: {e}"
    except KeyError:
        # Capturar errores si la estructura de respuesta no es la esperada
        return "Error: respuesta inesperada de la API."

@app.route("/chat", methods=["POST"])
def chat():
    """
    Endpoint para gestionar el chat.
    """
    data = request.json
    user_id = data.get("user_id")
    message = data.get("message")

    if not user_id or not message:
        return jsonify({"error": "user_id y message son requeridos"}), 400

    # Crear o recuperar el contexto del usuario
    if user_id not in user_sessions:
        user_sessions[user_id] = {"context": []}

    context = user_sessions[user_id]["context"]
    full_prompt = f"Contexto previo: {' '.join(context)}\nUsuario: {message}"

    # Generar respuesta usando la API de Mistral
    response = query_mistral(full_prompt)

    # Actualizar el contexto
    context.append(message)
    user_sessions[user_id]["context"] = context

    return jsonify({"response": response})

# Iniciar el servidor
def run():
    app.run(host="0.0.0.0", port=5002)  # Cambia a un puerto diferente si es necesario

# Iniciar el servidor en un hilo para evitar bloqueos
thread = Thread(target=run)
thread.start()


Ingresa tu API key de Mistral: ··········
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://172.28.0.12:5002
INFO:werkzeug:Press CTRL+C to quit


In [14]:
import requests

API_URL = "http://127.0.0.1:5002/chat"
user_id = "user123"

while True:
    message = input("Tú: ")
    if message.lower() == "salir":
        break
    response = requests.post(API_URL, json={"user_id": user_id, "message": message})
    if response.status_code == 200:
        print("Chatbot:", response.json()["response"])
    else:
        print("Error:", response.text)


Tú: Hola


INFO:werkzeug:127.0.0.1 - - [31/Dec/2024 00:24:27] "POST /chat HTTP/1.1" 200 -


Payload enviado: {'model': 'mistral-7b-v0.1', 'messages': [{'role': 'user', 'content': 'Contexto previo: \nUsuario: Hola'}], 'max_tokens': 150, 'temperature': 0.7, 'top_p': 1.0}
Respuesta completa del error: Sin respuesta del servidor
Chatbot: Error al consultar la API de Mistral: 400 Client Error: Bad Request for url: https://api.mistral.ai/v1/chat/completions
Tú: Mi nombre es david


INFO:werkzeug:127.0.0.1 - - [31/Dec/2024 00:24:36] "POST /chat HTTP/1.1" 200 -


Payload enviado: {'model': 'mistral-7b-v0.1', 'messages': [{'role': 'user', 'content': 'Contexto previo: Hola\nUsuario: Mi nombre es david'}], 'max_tokens': 150, 'temperature': 0.7, 'top_p': 1.0}
Respuesta completa del error: Sin respuesta del servidor
Chatbot: Error al consultar la API de Mistral: 400 Client Error: Bad Request for url: https://api.mistral.ai/v1/chat/completions
Tú: salir
